In [1]:
!pip install -q \
    transformers \
    accelerate \
    peft \
    bitsandbytes \
    datasets \
    sentence-transformers \
    faiss-cpu \
    pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 33.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import torch
import faiss
import json
import numpy as np
import fitz
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model

2025-12-18 19:17:24.470021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766085444.660397      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766085444.714840      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766085445.168683      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766085445.168724      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766085445.168727      55 computation_placer.cc:177] computation placer alr

In [3]:
MODEL_NAME = "Qwen/Qwen3-4B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [4]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,949,120 || all params: 4,025,417,216 || trainable%: 0.0733


In [6]:
dataset = load_dataset(
    "json",
    data_files="/kaggle/input/lora-train/lora_train.jsonl"
)

def tokenize(example):
    messages = [
        {"role": "user", "content": example["instruction"]},
        {"role": "assistant", "content": example["response"]}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    out = tokenizer(
        text,
        truncation=True,
        max_length=512
    )
    out["labels"] = out["input_ids"].copy()
    return out

dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [7]:
args = TrainingArguments(
    output_dir="lora_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    logging_steps=10,
    save_strategy="epoch",
    optim="paged_adamw_8bit",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"]
)

trainer.train()

Step,Training Loss
10,1.912800
20,1.646700
30,1.604000
40,1.434800
50,1.496900
60,1.344100


TrainOutput(global_step=64, training_loss=1.5656195431947708, metrics={'train_runtime': 172.5021, 'train_samples_per_second': 1.484, 'train_steps_per_second': 0.371, 'total_flos': 2236379911907328.0, 'train_loss': 1.5656195431947708, 'epoch': 2.0})

In [8]:
model.save_pretrained("qwen3-4b-course")
tokenizer.save_pretrained("qwen3-4b-course")

('qwen3-4b-course/tokenizer_config.json',
 'qwen3-4b-course/special_tokens_map.json',
 'qwen3-4b-course/chat_template.jinja',
 'qwen3-4b-course/vocab.json',
 'qwen3-4b-course/merges.txt',
 'qwen3-4b-course/added_tokens.json',
 'qwen3-4b-course/tokenizer.json')

In [9]:
index = faiss.read_index("/kaggle/input/vectorstore/index.faiss")
texts = np.load("/kaggle/input/vectorstore/texts.npy", allow_pickle=True)

embedder = SentenceTransformer("intfloat/e5-base-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [10]:
!pip install -q \
    arxiv \
    youtube-search

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.3 MB/s eta 0:00:00


In [12]:
import sys
import arxiv
import subprocess
from youtube_search import YoutubeSearch

def extract_concepts(question):
    """Isolates core technical keywords from the question."""
    # This prompt provides an example to guide the model's output format
    extraction_prompt = f"""
Extract 2-3 core mathematical concepts from the question below.
Example: 
Question: "Calculate the PDF of Z = X + Y for Gaussian variables."
Keywords: Gaussian distribution, Probability Density Function

Question: "{question}"
Keywords:"""
    
    inputs = tokenizer(extraction_prompt, return_tensors="pt").to(model.device)
    
    # max_new_tokens=30 keeps the output short to prevent instructions from leaking
    output = model.generate(
        **inputs, 
        max_new_tokens=30, 
        temperature=0.1, 
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )
    
    full_text = tokenizer.decode(output[0], skip_special_tokens=True)
    
    # Split by 'Keywords:' and take only the first line to discard extra model chatter
    if "Keywords:" in full_text:
        concepts = full_text.split("Keywords:")[-1].strip().split('\n')[0]
    else:
        concepts = full_text.split('\n')[-1].strip()
        
    return concepts

def youtube_search(query, max_results=3):
    """Fetches video links using the youtube-search library."""
    try:
        # We clean the query to ensure it's just words separated by spaces
        clean_query = query.replace(",", " ")
        results = YoutubeSearch(clean_query, max_results=max_results).to_dict()
        
        links = []
        for v in results:
            links.append({
                "title": v['title'],
                "link": f"https://www.youtube.com{v['url_suffix']}"
            })
        return links
    except Exception as e:
        print(f"YT Search Error: {e}")
        return []
def arxiv_search(query, max_results=3):
    try:
        client = arxiv.Client()
        search_term = query.replace(",", " ")
        search = arxiv.Search(query=search_term, max_results=max_results)
        return [{"title": res.title, "url": res.pdf_url} for res in client.results(search)]
    except:
        return []

In [13]:
def retrieve_context(question, k=4):
    emb = embedder.encode([question]).astype("float32")
    _, idx = index.search(emb, k)
    return "\n\n".join(texts[i] for i in idx[0])

In [24]:
def generate_answer(question):
    # STEP 1: Concept Extraction
    concepts = extract_concepts(question)
    search_query = concepts.replace(",", " ")
    
    context = retrieve_context(question)

    messages = [
        {
            "role": "system",
            "content": "You are an IIIT course assistant."
        },
        {
            "role": "user",
            "content": f"""
                Context:
                {context}
                
                Question:
                {question}
                
                Think briefly, then answer. Answer clearly, completely, briefly and in a structured manner.
                """
        }
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=7500,
        temperature=0.6,
        top_p=0.95
    )

    raw_answer = tokenizer.decode(output[0], skip_special_tokens=True)
    thoughts = raw_answer.split("assistant")[-1].split("</think>")[0].strip()
    clean_answer = raw_answer.split("assistant")[-1].split("</think>")[-1].strip()

    print("🌐 Fetching external resources...")
    sys.stdout.flush()
    
    yt_links = youtube_search(search_query)
    arxiv_links = arxiv_search(search_query)

    return {
        "extracted_concepts": concepts,
        "search_query": search_query,
        "thoughts": thoughts,
        "answer": clean_answer,
        "youtube": yt_links,
        "papers": arxiv_links
    }

# Validation

In [23]:
# 🔍 Trial test cell — verify RAG + LoRA model works end-to-end
test_question = "What is a random variable? Explain with examples."

print("=" * 70)
print(" IIIT Course Assistant – Trial RAG Test")
print("=" * 70)

print("\n🧑‍🎓 Question:")
print(test_question)

print("\n🔍 Thinking...\n")

result = generate_answer(test_question)

print(f"\n🧠 Model Extraction: {result['extracted_concepts']}")
print(f"🔍 Query fed to YT/ArXiv: {result['search_query']}")
print("-" * 70)

print("\n📘 Assistant Thoughts:")
thoughts = result["thoughts"].strip()
print(thoughts)

print("-" * 70)

print("\n📘 Assistant Answer:")
# Cleaning the answer display to remove the chat history prefix if needed
final_answer = result["answer"].split("assistant")[-1].strip()
print(final_answer)

print("-" * 70)

if result.get("youtube"):
    print("\n📺 Suggested YouTube Videos:")
    for vid in result["youtube"]:
        print(f"  • {vid['title']}: {vid['link']}")

if result.get("papers"):
    print("\n📄 Relevant Research Papers (arXiv):")
    for paper in result["papers"]:
        print(f"  • {paper['title']}: {paper['url']}")

print("\n" + "=" * 70)
print("✅ Test completed with concept-aware search.")

 IIIT Course Assistant – Trial RAG Test

🧑‍🎓 Question:
What is a random variable? Explain with examples.

🔍 Thinking...

🌐 Fetching external resources...

🧠 Model Extraction: Discrete random variable, Continuous
🔍 Query fed to YT/ArXiv: Discrete random variable  Continuous
----------------------------------------------------------------------

📘 Assistant Thoughts:
<think>
Okay, the user is asking about random variables, specifically what they are and examples. Let me start by recalling the definitions provided in the context. The context mentions that a random variable is discrete if its CDF is piecewise constant and continuous if the CDF is a continuous function. Also, there's an example with a running experiment involving a coin and a die.

First, I need to define a random variable. From the context, a random variable is a function that assigns a real number to each outcome in the sample space. The key is that it's a function from the sample space to the real numbers. 

Next, I shou

In [28]:
import json
import fitz
import re

PDF_PATH = "/kaggle/input/endsem/endsem_solutions.pdf"

def extract_questions_from_solutions(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text()

    question_blocks = re.split(r'(Q\d+[:])', full_text)

    extracted_questions = []
    for i in range(1, len(question_blocks), 2):
        q_id = question_blocks[i].replace(":", "").strip()   # Q1
        q_content = question_blocks[i+1]
        question_text = q_content.split('A:')[0].strip()
        extracted_questions.append((q_id, question_text))

    return extracted_questions


print("📄 Reading questions from endsem PDF...")
questions = extract_questions_from_solutions(PDF_PATH)

print(f"✅ Found {len(questions)} questions. Starting evaluation...\n")
print("Questions found: ")
print(questions)
print()
results = {}
id = 1

for q_id, q_text in questions:
    q_id = id
    print(f"🧪 Evaluating {q_id}...")
    try:
        ans = generate_answer(q_text)

        results[q_id] = {
            "question_id": q_id,
            "input_question": q_text,
            "retrieval": {
                "extracted_concepts": ans["extracted_concepts"],
                "search_query": ans["search_query"]
            },
            "model_response": {
                "answer": ans["answer"].split("assistant")[-1].strip(),
                "thoughts": ans["thoughts"].strip()
            },
            "resources": {
                "youtube": ans["youtube"],
                "papers": ans["papers"]
            }
        }

    except Exception as e:
        results[q_id] = {
            "question_id": q_id,
            "input_question": q_text,
            "error": str(e)
        }
    id += 1

# Save final results
output_file = "endsem_results.json"
with open(output_file, "w") as f:
    json.dump(results, f, indent=4)

print(f"\n🎯 Endsem evaluation complete.")
print(f"📁 Results saved to {output_file}")

📄 Reading questions from endsem PDF...
✅ Found 12 questions. Starting evaluation...

Questions found: 
[('Q1', 'Let Z = X1 + X2 + · · · + XN, where Xi are i.i.d. random variables and N is a\npositive discrete random variable. Prove that:\nMZ(t) = MN(log MX(t)).'), ('Q2', 'Let Y = eX, where X ∼N(µ, σ2). Obtain the pdf of Y .'), ('Q1', 'Let D = {x1, x2, . . . , xn} denote i.i.d. samples from a Poisson random variable with\nunknown parameter γ.\n(a) Find the Maximum Likelihood Estimate (MLE) for the unknown parameter\nγ. (5 marks)\n(b) Determine the Mean Squared Error (MSE) of the estimate. (3 marks)'), ('Q2', 'Consider a Gaussian random variable X with a known mean µ but an unknown\nvariance σ2. Suppose you observe k iid samples from this random variable, denoted\nby D = {x1, x2, . . . , xk}.\n(a) Find the MLE for the unknown variance σ2.\n(5 marks)\n(b) Is the MLE estimate biased? Justify your answer.\n(3 marks)'), ('Q3', 'Consider a sequence {Xn, n = 1, 2, 3, . . . } such that\nXn =\n(

In [29]:
!zip -r working.zip /kaggle/working

updating: kaggle/working/ (stored 0%)
updating: kaggle/working/.virtual_documents/ (stored 0%)
updating: kaggle/working/.virtual_documents/__notebook_source__.ipynb (deflated 64%)
updating: kaggle/working/lora_out/ (stored 0%)
updating: kaggle/working/lora_out/checkpoint-32/ (stored 0%)
updating: kaggle/working/lora_out/checkpoint-32/scaler.pt (deflated 64%)
updating: kaggle/working/lora_out/checkpoint-32/training_args.bin (deflated 53%)
updating: kaggle/working/lora_out/checkpoint-32/trainer_state.json (deflated 61%)
updating: kaggle/working/lora_out/checkpoint-32/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
updating: kaggle/working/lora_out/checkpoint-32/README.md (deflated 65%)
updating: kaggle/working/lora_out/checkpoint-32/scheduler.pt (deflated 61%)
updating: kaggle/working/lora_out/checkpoint-32/rng_state.pth (deflated 26%)
updating: kaggle/working/lora_out/checkpoint-32/adapter_config.json (deflated 56%)
updating: kaggle/working/lora_out/checkpoint-32/optimizer.pt (deflated 7%)
updating: kaggle/working/lora_out/checkpoint-64/ (stored 0%)
updating: kaggle/working/lora_out/checkpoint-64/scaler.pt (deflated 64%)
updating: kaggle/working/lora_out/checkpoint-64/training_args.bin (deflated 53%)
updating: kaggle/working/lora_out/checkpoint-64/trainer_state.json (deflated 66%)
updating: kaggle/working/lora_out/checkpoint-64/adapter_model.safetensors (deflated 8%)
updating: kaggle/working/lora_out/checkpoint-64/README.md (deflated 65%)
updating: kaggle/working/lora_out/checkpoint-64/scheduler.pt (deflated 62%)
updating: kaggle/working/lora_out/checkpoint-64/rng_state.pth (defl